In [1]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [2]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-2945527f-593b-4959-a78d-ac314fb6914f'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [3]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'Kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [4]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match("EUROPE", matchId)
    if match_detail["info"]["mapId"] in [1, 2, 11]:
        match = {}
        match["matchId"] = matchId
        match["gameDuration"] = match_detail["info"]["gameDuration"]
        match["gameType"] = match_detail["info"]["gameType"]
        match["gameVersion"] = match_detail["info"]["gameVersion"]
        matches.append(match)
        for index, row in enumerate(match_detail["info"]["participants"]):
            participants_row = {}
            participants_row["matchId"] = matchId
            participants_row["participant"] = index + 1
            participants_row["summonerName"] = row["summonerName"]
            participants_row["teamPosition"] = row["teamPosition"]
            participants_row["championName"] = row["championName"]
            participants_row["kda"] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
            participants_row["champLevel"] = row["champLevel"]
            participants_row["totalDamageToChampions"] = row[
                "totalDamageDealtToChampions"
            ]
            participants_row["totalDamageTaken"] = row["totalDamageTaken"]
            participants_row["totalHeal"] = row["totalHeal"]
            participants_row["totalMinionsKilled"] = row["totalMinionsKilled"]
            participants_row["goldEarned"] = row["goldEarned"]
            participants_row["summoner1"] = row["summoner1Id"]
            participants_row["summoner2"] = row["summoner2Id"]
            participants_row["item0"] = row["item0"]
            participants_row["item1"] = row["item1"]
            participants_row["item2"] = row["item2"]
            participants_row["item3"] = row["item3"]
            participants_row["item4"] = row["item4"]
            participants_row["item5"] = row["item5"]
            participants_row["item6"] = row["item6"]
            participants_row["win"] = row["win"]
            participants.append(participants_row)

        for row in timeline["info"]["frames"]:
            # frameCounter = 1
            # frame = row["{:02d}".format(frameCounter)]
            # frameCounter += 1
            for frameRow in row["events"]:
                timeline_row = {}
                timeline_row["matchId"] = matchId
                eventCounter = 0
                # eventFrame = frameRow["{:02d}".format(eventCounter)]
                eventFrame = frameRow
                eventCounter += 1

                if eventFrame["type"] == "CHAMPION_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["victimId"] = eventFrame["victimId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    if "bounty" in eventFrame:
                        timeline_row["bounty"] = eventFrame["bounty"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "BUILDING_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["teamId"]
                    timeline_row["buildingType"] = eventFrame["buildingType"]
                    timeline_row["laneType"] = eventFrame["laneType"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ITEM_PURCHASED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["itemId"] = eventFrame["itemId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_PLACED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["creatorId"] = eventFrame["creatorId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "LEVEL_UP":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["level"] = eventFrame["level"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ELITE_MONSTER_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["killerTeamId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    timeline_row["monsterType"] = eventFrame["monsterType"]
                    matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
matchTimelineDf["timestampMIN"] = round(matchTimelineDf["timestamp"] / 60000, 0)


In [51]:
match_detail["info"]["mapId"]

12

In [5]:
participantsDf

,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5747889998,1,TrouserLeviathan,TOP,MonkeyKing,16/8/8,18,42880,48128,16089,...,14,12,4401,3111,6632,3071,6333,3053,3340,True
1,EUW1_5747889998,2,FlaviüsJustinian,JUNGLE,Rammus,4/7/21,16,18019,37298,13832,...,4,11,3111,6664,3075,3742,1057,1028,3364,True
2,EUW1_5747889998,3,Mindusia1,MIDDLE,Orianna,6/7/14,16,21656,22996,2521,...,12,4,4644,4645,3135,1058,0,3020,3340,True
3,EUW1_5747889998,4,GranpaLongShlong,BOTTOM,MissFortune,15/12/9,16,33222,28844,2052,...,3,4,1055,6676,3006,6671,3814,3036,3363,True
4,EUW1_5747889998,5,ConfeTea,UTILITY,Thresh,2/9/17,14,8844,28523,3736,...,14,4,3857,1028,3190,3075,3067,3111,3364,True
5,EUW1_5747889998,6,NigerianPrinceEU,TOP,Nidalee,2/10/10,16,13491,34092,7849,...,12,4,6662,3121,3157,3047,1056,3108,3363,False
6,EUW1_5747889998,7,NightWõlf,JUNGLE,Karthus,16/6/14,18,57759,30565,10335,...,4,11,3041,3135,6653,3020,3089,4645,3340,False
7,EUW1_5747889998,8,Kelody,MIDDLE,Akali,17/9/6,15,33219,36365,4984,...,4,12,1058,3152,3157,4645,3020,3135,3363,False
8,EUW1_5747889998,9,Sir Nurum,BOTTOM,Ashe,5/11/14,15,21402,26035,1672,...,7,4,1055,6672,1036,3006,3085,3153,3340,False
9,EUW1_5747889998,10,Fegel,UTILITY,Braum,3/7/20,13,10975,28268,2627,...,14,4,3047,2055,3190,3075,3067,3860,3364,False


In [9]:
matchTimelineDf["assistingParticipantIds"] = [ [] if x is np.NaN else x for x in matchTimelineDf["assistingParticipantIds"]]

In [10]:
def getSummonerName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = int(row["killerId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = int(row["victimId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def getChampionName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = row["killerId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = row["victimId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def checkAssist(row, name):
    matchId = row["matchId"]
    if row["assistingParticipantIds"]:
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0] == name:
                return True
    return False


In [11]:
# matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
# matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
# matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)

0       False
1       False
2       False
3       False
4       False
        ...  
3437    False
3438    False
3439    False
3440    False
3441    False
Length: 3442, dtype: bool

In [12]:
matchTimelineDf["participantName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
matchTimelineDf["participantChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "participant"), axis=1)
matchTimelineDf["killerName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
matchTimelineDf["killerChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "killer"), axis=1)
matchTimelineDf["victimName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf["victimChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "victim"), axis=1)
matchTimelineDf["kelodyAssist"] = matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)


In [13]:
matchTimelineDf

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5747889998,ITEM_PURCHASED,10.0,3858.0,4215,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Fegel,Braum,NaN,NaN,NaN,NaN,False
1,EUW1_5747889998,ITEM_PURCHASED,10.0,2003.0,4215,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Fegel,Braum,NaN,NaN,NaN,NaN,False
2,EUW1_5747889998,ITEM_PURCHASED,10.0,2003.0,4215,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Fegel,Braum,NaN,NaN,NaN,NaN,False
3,EUW1_5747889998,ITEM_PURCHASED,10.0,3340.0,4810,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Fegel,Braum,NaN,NaN,NaN,NaN,False
4,EUW1_5747889998,ITEM_PURCHASED,7.0,1035.0,8906,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NightWõlf,Karthus,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3437,EUW1_5743980122,ITEM_PURCHASED,7.0,1058.0,2436237,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,41.0,QuoIKkA,Diana,NaN,NaN,NaN,NaN,False
3438,EUW1_5743980122,CHAMPION_KILL,NaN,NaN,2436700,NaN,NaN,NaN,1.0,6.0,...,NaN,NaN,41.0,NaN,NaN,Elite25O,Vladimir,NightWõlf,Tryndamere,False
3439,EUW1_5743980122,ITEM_PURCHASED,4.0,2140.0,2436799,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,41.0,Adromas,Zeri,NaN,NaN,NaN,NaN,False
3440,EUW1_5743980122,BUILDING_KILL,NaN,NaN,2439541,NaN,NaN,NaN,3.0,NaN,...,TOWER_BUILDING,MID_LANE,41.0,NaN,NaN,KC Kant,Talon,NaN,NaN,False


In [14]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
7,EUW1_5747889998,8
17,EUW1_5747855273,8
27,EUW1_5745362126,8
37,EUW1_5745318060,8
47,EUW1_5744361064,8
57,EUW1_5743980122,8


In [15]:
kelodyEvents = []
#TODO bool if kelody was assist or not
# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5747889998,ITEM_PURCHASED,8.0,1054.0,33582,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,Kelody,Akali,NaN,NaN,NaN,NaN,False
1,EUW1_5747889998,ITEM_PURCHASED,8.0,2003.0,33582,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,Kelody,Akali,NaN,NaN,NaN,NaN,False
2,EUW1_5747889998,LEVEL_UP,8.0,NaN,130015,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,2.0,Kelody,Akali,NaN,NaN,NaN,NaN,False
3,EUW1_5747889998,LEVEL_UP,8.0,NaN,158291,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,3.0,Kelody,Akali,NaN,NaN,NaN,NaN,False
4,EUW1_5747889998,LEVEL_UP,8.0,NaN,195062,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,3.0,Kelody,Akali,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,EUW1_5743980122,ITEM_PURCHASED,8.0,3363.0,2307924,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,38.0,Kelody,Kassadin,NaN,NaN,NaN,NaN,False
376,EUW1_5743980122,WARD_PLACED,NaN,NaN,2359299,BLUE_TRINKET,8.0,NaN,NaN,NaN,...,NaN,NaN,39.0,NaN,NaN,NaN,NaN,NaN,NaN,False
377,EUW1_5743980122,CHAMPION_KILL,NaN,NaN,2401519,NaN,NaN,NaN,8.0,4.0,...,NaN,NaN,40.0,NaN,NaN,Kelody,Kassadin,Adromas,Zeri,False
378,EUW1_5743980122,CHAMPION_KILL,NaN,NaN,2409941,NaN,NaN,NaN,8.0,5.0,...,NaN,NaN,40.0,NaN,NaN,Kelody,Kassadin,UŁTRĄ INSTINCT,Alistar,False


In [16]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == True]

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
24,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,645689,NaN,NaN,NaN,7.0,5.0,...,NaN,NaN,11.0,NaN,NaN,NightWõlf,Karthus,ConfeTea,Thresh,True
36,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,820836,NaN,NaN,NaN,10.0,4.0,...,NaN,NaN,14.0,NaN,NaN,Fegel,Braum,GranpaLongShlong,MissFortune,True
43,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,916773,NaN,NaN,NaN,7.0,5.0,...,NaN,NaN,15.0,NaN,NaN,NightWõlf,Karthus,ConfeTea,Thresh,True
44,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,919417,NaN,NaN,NaN,7.0,4.0,...,NaN,NaN,15.0,NaN,NaN,NightWõlf,Karthus,GranpaLongShlong,MissFortune,True
45,EUW1_5747889998,ELITE_MONSTER_KILL,NaN,NaN,934511,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,16.0,NaN,NaN,NightWõlf,Karthus,NaN,NaN,True
46,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,974344,NaN,NaN,NaN,7.0,1.0,...,NaN,NaN,16.0,NaN,NaN,NightWõlf,Karthus,TrouserLeviathan,MonkeyKing,True
62,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,1228414,NaN,NaN,NaN,10.0,4.0,...,NaN,NaN,20.0,NaN,NaN,Fegel,Braum,GranpaLongShlong,MissFortune,True
118,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1154602,NaN,NaN,NaN,6.0,5.0,...,NaN,NaN,19.0,NaN,NaN,Junipie,Illaoi,KG Maisen,Pyke,True
153,EUW1_5745362126,CHAMPION_KILL,NaN,NaN,920860,NaN,NaN,NaN,7.0,2.0,...,NaN,NaN,15.0,NaN,NaN,JensChristian,Shaco,Satan4,Nidalee,True
208,EUW1_5744361064,CHAMPION_KILL,NaN,NaN,604348,NaN,NaN,NaN,7.0,3.0,...,NaN,NaN,10.0,NaN,NaN,NightWõlf,Zed,MaxiMoonLight,Viktor,True


Check how often a Player died in Early Game (pre 10 mins)

In [17]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
7,EUW1_5747889998,8,0
17,EUW1_5747855273,8,2
27,EUW1_5745362126,8,3
37,EUW1_5745318060,8,1
47,EUW1_5744361064,8,0
57,EUW1_5743980122,8,1


#### TODO
##### Features:
- Died from ganks in laning phase pre 15 min (check ID from enemy Jungler)

- Died with ... Gold

- <s>count 1v1 kill/deaths</s>  
  
- Good roaming (assists pre 15 min)

- Warding (check ward placements pre 15 min) maybe in combination with ganks

- CS per min (if data available)

- KDA (kills and assists divided by deaths) als Output KDA String aus participantDf

- <s>Tower Kills/Assists</s> 

- <s>Objective Kills/Assists (Baron, Dragon, Herald)</s> 



##### Features on the tableau visualization:

- <s>Timeline</s>

- <s>Champion Icons</s>

- Damage Dealt Bar Chart



Check 1v1

In [18]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == False]

def check1v1(row, name):
    if not row["assistingParticipantIds"] and (row["victimName"] == name or row["killerName"] == name) and (row["type"] == "CHAMPION_KILL"):
        return True
    return False

kelodyEventsDf["1v1"] = kelodyEventsDf.apply(lambda row: check1v1(row, "Kelody"), axis = 1)
kelodyEventsDf[kelodyEventsDf["1v1"] == True]

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,...,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist,1v1
9,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,358880,NaN,NaN,NaN,8.0,3.0,...,NaN,6.0,NaN,NaN,Kelody,Akali,Mindusia1,Orianna,False,True
14,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,451477,NaN,NaN,NaN,8.0,3.0,...,NaN,8.0,NaN,NaN,Kelody,Akali,Mindusia1,Orianna,False,True
23,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,571269,NaN,NaN,NaN,8.0,3.0,...,NaN,10.0,NaN,NaN,Kelody,Akali,Mindusia1,Orianna,False,True
52,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,1019597,NaN,NaN,NaN,8.0,5.0,...,NaN,17.0,NaN,NaN,Kelody,Akali,ConfeTea,Thresh,False,True
65,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,1286165,NaN,NaN,NaN,8.0,3.0,...,NaN,21.0,NaN,NaN,Kelody,Akali,Mindusia1,Orianna,False,True
76,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,1535905,NaN,NaN,NaN,1.0,8.0,...,NaN,26.0,NaN,NaN,TrouserLeviathan,MonkeyKing,Kelody,Akali,False,True
96,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,284035,NaN,NaN,NaN,3.0,8.0,...,NaN,5.0,NaN,NaN,r3dcactus,Neeko,Kelody,Ahri,False,True
104,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,512048,NaN,NaN,NaN,3.0,8.0,...,NaN,9.0,NaN,NaN,r3dcactus,Neeko,Kelody,Ahri,False,True
107,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,652234,NaN,NaN,NaN,3.0,8.0,...,NaN,11.0,NaN,NaN,r3dcactus,Neeko,Kelody,Ahri,False,True
115,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1017707,NaN,NaN,NaN,5.0,8.0,...,NaN,17.0,NaN,NaN,KG Maisen,Pyke,Kelody,Ahri,False,True


In [19]:
def count1v1(row, name, killer):
  if killer:
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["killerName"] == name)])
  else: 
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["victimName"] == name)])

kelody["1v1Won"] = kelody.apply(lambda row: count1v1(row, "Kelody", True), axis=1)
kelody["1v1Lost"] = kelody.apply(lambda row: count1v1(row, "Kelody", False), axis=1)
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost
7,EUW1_5747889998,8,0,5,1
17,EUW1_5747855273,8,2,0,6
27,EUW1_5745362126,8,3,2,4
37,EUW1_5745318060,8,1,0,1
47,EUW1_5744361064,8,0,6,2
57,EUW1_5743980122,8,1,2,3


In [20]:
def countTurretKills(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "BUILDING_KILL") & (kelodyEventsDf["killerName"] == name)])

kelody["turretKills"] = kelody.apply(lambda row: countTurretKills(row, "Kelody"), axis=1)
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills
7,EUW1_5747889998,8,0,5,1,1
17,EUW1_5747855273,8,2,0,6,0
27,EUW1_5745362126,8,3,2,4,1
37,EUW1_5745318060,8,1,0,1,0
47,EUW1_5744361064,8,0,6,2,2
57,EUW1_5743980122,8,1,2,3,0


In [21]:
def counEliteMonsterKills(row, monsterName):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "ELITE_MONSTER_KILL") & (kelodyEventsDf["monsterType"] == monsterName)])

kelody["dragonKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "DRAGON"), axis=1)
kelody["baronKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "RIFTHERALD"), axis=1)
kelody["heraldKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "BARON_NASHOR"), axis=1)

kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills
7,EUW1_5747889998,8,0,5,1,1,1,0,0
17,EUW1_5747855273,8,2,0,6,0,0,0,0
27,EUW1_5745362126,8,3,2,4,1,1,0,0
37,EUW1_5745318060,8,1,0,1,0,0,0,0
47,EUW1_5744361064,8,0,6,2,2,2,0,0
57,EUW1_5743980122,8,1,2,3,0,0,0,0


In [66]:
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
kelodyEventsDf.to_json('kelodyEvents.json', orient='records')
kelody.to_json('kelodyStats.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
